In [ ]:
import os
from keras import Input, Model
from keras import optimizers, regularizers
from keras.callbacks import ModelCheckpoint
from keras.layers import (add,
                          Conv2D, GlobalAveragePooling2D)
from sklearn.model_selection import train_test_split
from tensorflow import keras
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.datasets import cifar10
from keras.layers import Dense, Activation, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import freezing

In [ ]:
# Training procedure for CIFAR-10 using ResNet 110.
# ResNet model from https://github.com/BIGBALLON/cifar-10-cnn/blob/master/4_Residual_Network/ResNet_keras.py
stack_n = 18
num_classes = 10
img_rows, img_cols = 32, 32
img_channels = 3
batch_size = 128
epochs = 200
iterations = 45000 // batch_size
weight_decay = 0.0001
seed = 333


def residual_network(img_input, classes_num=10, stack_n=5):
    def residual_block(intput, out_channel, increase=False):
        if increase:
            stride = (2, 2)
        else:
            stride = (1, 1)

        pre_bn = BatchNormalization()(intput)
        pre_relu = Activation('relu')(pre_bn)

        conv_1 = Conv2D(out_channel, kernel_size=(3, 3), strides=stride, padding='same',
                        kernel_initializer="he_normal",
                        kernel_regularizer=regularizers.l2(weight_decay))(pre_relu)
        bn_1 = BatchNormalization()(conv_1)
        relu1 = Activation('relu')(bn_1)
        conv_2 = Conv2D(out_channel, kernel_size=(3, 3), strides=(1, 1), padding='same',
                        kernel_initializer="he_normal",
                        kernel_regularizer=regularizers.l2(weight_decay))(relu1)
        if increase:
            projection = Conv2D(out_channel,
                                kernel_size=(1, 1),
                                strides=(2, 2),
                                padding='same',
                                kernel_initializer="he_normal",
                                kernel_regularizer=regularizers.l2(weight_decay))(intput)
            block = add([conv_2, projection])
        else:
            block = add([intput, conv_2])
        return block

    # build model
    # total layers = stack_n * 3 * 2 + 2
    # stack_n = 5 by default, total layers = 32
    # input: 32x32x3 output: 32x32x16
    x = Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), padding='same',
               kernel_initializer="he_normal",
               kernel_regularizer=regularizers.l2(weight_decay))(img_input)

    # input: 32x32x16 output: 32x32x16
    for _ in range(stack_n):
        x = residual_block(x, 16, False)

    # input: 32x32x16 output: 16x16x32
    x = residual_block(x, 32, True)
    for _ in range(1, stack_n):
        x = residual_block(x, 32, False)

    # input: 16x16x32 output: 8x8x64
    x = residual_block(x, 64, True)
    for _ in range(1, stack_n):
        x = residual_block(x, 64, False)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)

    # input: 64 output: 10
    x = Dense(classes_num, activation='softmax',
              kernel_initializer="he_normal",
              kernel_regularizer=regularizers.l2(weight_decay))(x)
    return x


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train45, x_val, y_train45, y_val = train_test_split(x_train, y_train, test_size=0.1,
                                                      random_state=seed)  # random_state = seed

img_mean = x_train45.mean(axis=0)  # per-pixel mean, what about std?
img_std = x_train45.std(axis=0)
x_train45 = (x_train45 - img_mean) / img_std
x_val = (x_val - img_mean) / img_std
x_test = (x_test - img_mean) / img_std

# build network
img_input = Input(shape=(img_rows, img_cols, img_channels))
output = residual_network(img_input, num_classes, stack_n)
model = Model(img_input, output)
print(model.summary())

# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant', cval=0.)

datagen.fit(x_train45)

In [3]:
freezing_list = []
for i in range(len(model.layers)):
    if i < len(model.layers) * 0.8:
        freezing_list.append(int(epochs * 0.6))
freezing_list.append(epochs)
checkpointer = ModelCheckpoint('model_resnet_c10_best.hdf5', verbose=1, save_best_only=True)

In [4]:
freezing.training_with_freezing(model, datagen, sgd, x_train45, y_train45, x_val, y_val, x_test, y_test, freezing_list,
                       batch_size=128, lr_schedule=[[0, 0.1], [80, 0.01], [150, 0.001]], cbks=[checkpointer],
                       name='resnet_cifar10')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_100
.........vars
......activation_101
.........vars
......activation_102
.........vars
......activation_103
.........vars
......activation_104
.........vars
......activation_105
.........vars
......activation_106
.........vars
......activation_107
.........vars
......activation_108
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
..

2023-04-21 10:23:43.603064: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200


351/351 [==============================] - ETA: 0s - loss: 2.2965 - accuracy: 0.4400
Epoch 1: val_loss improved from inf to 2.57642, saving model to model_resnet_c10_best.hdf5
351/351 [==============================] - 75s 140ms/step - loss: 2.2965 - accuracy: 0.4400 - val_loss: 2.5764 - val_accuracy: 0.4382 - lr: 0.1000
Epoch 2/120
351/351 [==============================] - ETA: 0s - loss: 1.7975 - accuracy: 0.6081
Epoch 2: val_loss improved from 2.57642 to 2.54349, saving model to model_resnet_c10_best.hdf5
351/351 [==============================] - 45s 128ms/step - loss: 1.7975 - accuracy: 0.6081 - val_loss: 2.5435 - val_accuracy: 0.4324 - lr: 0.1000
Epoch 3/120
351/351 [==============================] - ETA: 0s - loss: 1.5331 - accuracy: 0.6792
Epoch 3: val_loss improved from 2.54349 to 1.58750, saving model to model_resnet_c10_best.hdf5
351/351 [==============================] - 46s 130ms/step - loss: 1.5331 - accuracy: 0.6792 - val_loss: 1.5875 - val_accuracy: 0.6522 - lr: 0.1000

[7.150000000000006,
 0.05053877526223651,
 0.7344198822975159,
 0.3587348235516382,
 0.062430451437824386]